In [1]:
import os

In [2]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import os
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2
import json


In [3]:
load_dotenv()

True

In [4]:
key=os.getenv("OpenAI_API_KEY")

In [5]:
llm=ChatOpenAI(openai_api_key=key ,model="gpt-3.5-turbo" , temperature=0.7)

In [6]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000029007FE7FD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000029008016D70>, root_client=<openai.OpenAI object at 0x000002907D3DBF70>, root_async_client=<openai.AsyncOpenAI object at 0x0000029007FE7DF0>, temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [7]:
with open("D:\mcq generator\mcq generator app\Response.json","r" ) as f:
    RESPONSE_JSON = json.load(f)

In [8]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '4': {'no': '4', 'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [18]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [19]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [20]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [21]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [22]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key='review',verbose=True)

In [23]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [24]:
 PATH="D:\mcq generator\mcq generator app\data.txt"

In [25]:
print(PATH)

D:\mcq generator\mcq generator app\data.txt


In [26]:
with open(PATH,"r") as f:
    text=f.read()

In [27]:
print(text)

OpenAI, Inc. is an American artificial intelligence (AI) research organization founded in December 2015 and headquartered in San Francisco, California. It aims to develop "safe and beneficial" artificial general intelligence (AGI), which it defines as "highly autonomous systems that outperform humans at most economically valuable work".[6] As a leading organization in the ongoing AI boom,[7] OpenAI is known for the GPT family of large language models, the DALL-E series of text-to-image models, and a text-to-video model named Sora.[8][9] Its release of ChatGPT in November 2022 has been credited with catalyzing widespread interest in generative AI.

The organization has a complex corporate structure. As of April 2025, it is led by the non-profit OpenAI, Inc.,[1] registered in Delaware, and has multiple for-profit subsidiaries including OpenAI Holdings, LLC and OpenAI Global, LLC.[10] Microsoft has invested US$13 billion in OpenAI, and is entitled to 49% of OpenAI Global, LLC's profits, c

In [28]:
generate_evaluate_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{RESPONSE_JSON}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000029007FE7FD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000029008016D70>, root_client=<openai.OpenAI object at 0x000002907D3DBF70>, root_async_client=<openai.AsyncOpenAI object at 0x0000029007FE7DF0>, temperature=0

In [29]:
text
NUMBER=5
SUBJECT="OPENAI"
TONE="SIMPLE"
RESPONSE_JSON=RESPONSE_JSON


In [30]:
RESPONSE_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '4': {'no': '4',
  'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [31]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"no": "4", "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [33]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text": text,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
        
    )
    



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:OpenAI, Inc. is an American artificial intelligence (AI) research organization founded in December 2015 and headquartered in San Francisco, California. It aims to develop "safe and beneficial" artificial general intelligence (AGI), which it defines as "highly autonomous systems that outperform humans at most economically valuable work".[6] As a leading organization in the ongoing AI boom,[7] OpenAI is known for the GPT family of large language models, the DALL-E series of text-to-image models, and a text-to-video model named Sora.[8][9] Its release of ChatGPT in November 2022 has been credited with catalyzing widespread interest in generative AI.

The organization has a complex corporate structure. As of April 2025, it is led by the non-profit OpenAI, Inc.,[1] registered in Delaware, and has multiple for-profit subsidiaries including OpenAI Holdings, LLC and OpenAI Global, LLC.[1

In [34]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")


Total Tokens:2155
Prompt Tokens:1378
Completion Tokens:777
Total Cost:0.0018544999999999998


In [35]:
print(response)

{'text': 'OpenAI, Inc. is an American artificial intelligence (AI) research organization founded in December 2015 and headquartered in San Francisco, California. It aims to develop "safe and beneficial" artificial general intelligence (AGI), which it defines as "highly autonomous systems that outperform humans at most economically valuable work".[6] As a leading organization in the ongoing AI boom,[7] OpenAI is known for the GPT family of large language models, the DALL-E series of text-to-image models, and a text-to-video model named Sora.[8][9] Its release of ChatGPT in November 2022 has been credited with catalyzing widespread interest in generative AI.\n\nThe organization has a complex corporate structure. As of April 2025, it is led by the non-profit OpenAI, Inc.,[1] registered in Delaware, and has multiple for-profit subsidiaries including OpenAI Holdings, LLC and OpenAI Global, LLC.[10] Microsoft has invested US$13 billion in OpenAI, and is entitled to 49% of OpenAI Global, LLC\

In [36]:
quiz=response.get("quiz")

In [37]:
quiz =json.loads(quiz)

In [38]:
quiz

{'1': {'no': '1',
  'mcq': 'What is the main goal of OpenAI?',
  'options': {'a': 'To develop safe and beneficial artificial general intelligence',
   'b': 'To outperform humans in all tasks',
   'c': 'To dominate the AI industry',
   'd': 'To create entertainment AI models'},
  'correct': 'a'},
 '2': {'no': '2',
  'mcq': 'Which of the following is NOT a product known to be developed by OpenAI?',
  'options': {'a': 'GPT language models',
   'b': 'DALL-E text-to-image models',
   'c': 'Sora text-to-video model',
   'd': 'VR gaming software'},
  'correct': 'd'},
 '3': {'no': '3',
  'mcq': 'In which state is OpenAI, Inc. registered as a non-profit organization?',
  'options': {'a': 'California',
   'b': 'New York',
   'c': 'Delaware',
   'd': 'Texas'},
  'correct': 'c'},
 '4': {'no': '4',
  'mcq': 'Which company has invested $13 billion in OpenAI and provides computing resources through Microsoft Azure?',
  'options': {'a': 'Google', 'b': 'Apple', 'c': 'Amazon', 'd': 'Microsoft'},
  'corr

In [39]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [40]:
quiz_table_data

[{'MCQ': 'What is the main goal of OpenAI?',
  'Choices': 'a: To develop safe and beneficial artificial general intelligence | b: To outperform humans in all tasks | c: To dominate the AI industry | d: To create entertainment AI models',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a product known to be developed by OpenAI?',
  'Choices': 'a: GPT language models | b: DALL-E text-to-image models | c: Sora text-to-video model | d: VR gaming software',
  'Correct': 'd'},
 {'MCQ': 'In which state is OpenAI, Inc. registered as a non-profit organization?',
  'Choices': 'a: California | b: New York | c: Delaware | d: Texas',
  'Correct': 'c'},
 {'MCQ': 'Which company has invested $13 billion in OpenAI and provides computing resources through Microsoft Azure?',
  'Choices': 'a: Google | b: Apple | c: Amazon | d: Microsoft',
  'Correct': 'd'},
 {'MCQ': 'Why did OpenAI face lawsuits in 2023 and 2024?',
  'Choices': 'a: For trademark infringement | b: For alleged copyright infringem

In [42]:
df =pd.DataFrame(quiz_table_data)

In [44]:
df.to_csv('Openai.csv',index=False)